In [99]:
from django.test import TestCase
# from .choices import *
import openai
from openai import OpenAI
import  choices
import pandas as pd
import numpy as np

# OpenAI key

In [100]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Get_headline_examples function

In [101]:
def get_headline_examples(selected_style):
    examples = {'Forward reference': ['Kinas Økonomiske Fremtid: Usikkerheten som Følger Etter Gullalderen', 'Denne appen endrer måten nordmenn handler mat på', 'Se prisforskjellen som overrasker mellom EX90 og XC90'],
                'Metaphor': ['Kinas Økonomiske Skute i Uvær: Rentekutt som Redningsflåte', 'Frifor: Navigatoren i Ultrabehandlet Matens Labyrint', 'Fra Dinosaur til Lyn: Volvos Elektriske Revolusjon Ruller Inn'],
                'Question': ['Kan Volvos EX90 virkelig spare deg for 293.000 kroner?', 'Er Kinas økonomiske gullalder over for godt?', 'Kan En App Revolusjonere Måten Vi Velger Mat På?']
                }
    
    return examples [selected_style]

# Generate_headlines function

In [102]:
def generate_headlines(headline_style, article_text, type_of_hdl):
    prompt = ""
    msg = ""
    examples= ""
    
    examples = get_headline_examples(headline_style)
    print("Style manipulation = ", headline_style )
    
    style_user_prompt = f"""Create a headline for the provided article that is designed to maximize engagement and click-through rates. The user has expressed a preference for the headlines: {examples} which use the style: {headline_style}. Try to mimic the writing styles of these headlines to appeal to the user in the best possible way. Ensure that your headline captures the essence of the article while also applying a similar writing styles as the provided examples. Keep the headline to a maximum of 10 words. Article: {article_text}"""
    style_system_msg = "You are an AI skilled in generating headlines using different jornalistic headline styles. Your goal is to create engaging headlines in Norwegian language, adhering to specific guidelines designed to enhance curiosity and emotional engagement."
    
    
    boring_user_prompt = f"""Create a boring, non-styled, and matter-of-fact headline for the provided article:{article_text} reflecting the main point of the article in a straightforward manner"""
    boring_system_msg = "You are a text summarizer. Your task is to generate a title for a given article in Norwegian language. The title should be boring, non-styled, and matter-of-fact, reflecting the main point of the article in a straightforward manner. Avoid using emotional language, opinions, or embellishments."

    if type_of_hdl == 'styled':
        prompt = style_user_prompt
        msg = style_system_msg
        
    elif type_of_hdl == 'boring':
        prompt = boring_user_prompt
        msg = boring_system_msg
        
    
    try:
        # Adjusted temperature and top_p values as per the "Creative Writing" settings
        completion = openai.chat.completions.create(
            model="gpt-4-0125-preview",
            temperature=0.7,  # Adjust to requirements. Controls 'creativity'
            top_p=0.8,  # Adjust to requirements 'Controls sampling'
            messages=[
                {"role": "system", "content": msg},
                {"role": "user", "content": prompt}
            ]
        )

        # Extract the generated headline from the response
        generated_headline = completion.choices[0].message if completion.choices else "No headline generated."
        new_headline = generated_headline.content
        cleaned_headline = new_headline.strip('"')

        return cleaned_headline
    except Exception as e:
        print(f"Error generating headline: {e}")
        # return original_title  # Use original text as a fallback in case of an error


# Main frunction

In [116]:
# -------------------------- main --------------------------
df=pd.read_csv('../data/final_demo.csv',index_col=0)
df.shape

(5, 7)

In [117]:
manipulated_titles = []
boring_titles = []

for index, row in df.iterrows():
    
    headline_s = generate_headlines(row['style'], row['text'], 'styled')
    manipulated_titles.append(headline_s)
    
    headline_b = generate_headlines(row['style'], row['text'], 'boring')
    boring_titles.append(headline_b)
    
    
print (manipulated_titles)

df['manipulated_title'] = manipulated_titles
df['title'] = boring_titles

Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
Style manipulation =  Forward reference
['Støre dropper klimatoppmøtet: Et skritt tilbake for Norge?', 'Skolene Kapitulerer: Islam Tar Over i Klasserommet', 'Frp og Høyre Skyter Gullfuglen: Sjokkmålingen som Ryster Innlandet', 'Hva Koster Egentlig Innvandringen Norge?', 'Advarer mot nåtidsbriller i debatten om samenes fortid']


In [114]:
df.head(20)

,original_title,image_url,published_date,article_url,category,text,style,manipulated_title,title
aid,,,,,,,,,
1,- Det er så kjedelig,https://labrador-www.kk.no/images/82180361.jpg...,2024-11-11 18:00:00,https://www.kk.no/mote/det-er-sa-kjedelig/8199...,Business,Kjendismerker\nEr kjendisenes skjønnhetsmerker...,Forward reference,Er Kjendisenes Skjønnhetsmerker Pengene Verdt?...,Kjendisenes lansering av skjønnhetsmerker
3,Semb om trenerlegenden: – Det irriterer meg hv...,https://g.acdn.no/obscura/API/dynamic/r1/ece5/...,2024-11-11 06:39:14,https://www.tb.no/semb-om-trenerlegenden-det-i...,Business,ÅSGÅRDSTRAND (Nettavisen): – Jeg sto ikke med ...,Question,Vil Trumps valgseier endre global politikk igjen?,"Nils Johan Semb diskuterer fotball, politikk o..."
2,SoMe-spesialist,https://kampanje.com/contentassets/952a52b44fc...,2024-11-11 13:23:54,https://kampanje.com/jobb/Jobb-liste/2024/2024...,Business,Gjensidige er et ledende nordisk forsikringsse...,Metaphor,Gjensidige: Fyrtårnet i Forsikringshavets Fremtid,Gjensidige søker SoMe-spesialist for markedsfø...
10,Drapsgåten har splittet et helt lokalsamfunn,https://image.klikk.no/7470800.jpg?imageId=747...,2024-11-12 06:08:48,https://www.klikk.no/historie/krim/drapsgaten-...,Vær,FANT KJÆRESTEN: Karen Read ble tiltalt for dra...,Question,Ble Karen Read urettferdig anklaget for kjæres...,Karen Read frifunnet for drap på kjæresten ett...
7,Egil (27) skal redde Norge: – Haaland tjener s...,https://www.cdn.tv2.no/images?imageId=17163399...,2024-11-12 09:02:06,https://www.tv2.no/sport/egil-27-skal-redde-no...,Vær,Se Slovenia-Norge på TV 2 Direkte og Play tors...,Forward reference,Fra Haugesund til Helt: Selviks Uventede Reise...,Egil Selvik forbereder seg på landslagskamper ...
17,Bravida samler all aktivitet i Bergen i ikonis...,https://www.vvsaktuelt.no/bravida-samler-all-a...,2024-11-11 08:01:00,https://www.vvsaktuelt.no/bravida-samler-all-a...,Vær,"Bildet: Bravidas regiondirektør i Vestland, Gi...",Metaphor,Bravidas Fusjon Skaper Et Landemerke: Sammen u...,Bravida og Thunestvedt flytter til felles kont...


# Remember to remove the header to fill the data into the demo's databse

In [118]:
df.to_csv('../data/final_demo_manipulated.csv')